In [1]:
import sys
from pathlib import Path
from os.path  import join

# Add the project root to sys.path
project_root = Path().resolve().parent  # Adjust as needed to point to the root folder
sys.path.append(str(project_root))

print(Path.cwd())  # Check if the path is added


g:\Work\DS\mimolm\notebooks


In [2]:
from av2.datasets.motion_forecasting import scenario_serialization
from av2.map.map_api import ArgoverseStaticMap

In [32]:
scenario_path = "../data/argoex/0a0af725-fbc3-41de-b969-3be718f694e2/scenario_0a0af725-fbc3-41de-b969-3be718f694e2.parquet"
static_map_path = Path("../data/argoex/0a0af725-fbc3-41de-b969-3be718f694e2/log_map_archive_0a0af725-fbc3-41de-b969-3be718f694e2.json")

scenario = scenario_serialization.load_argoverse_scenario_parquet(scenario_path)
argomap = ArgoverseStaticMap.from_json(static_map_path)

visualize_scenario(scenario, argomap, Path("../data/argoex/argoext.mp4"))

In [42]:
scenario.tracks[3].object_states

[ObjectState(observed=True, timestep=0, position=(1515.091954808732, -1215.7317426738146), heading=2.777955112814176, velocity=(-12.291568400810661, 4.676217144421995)),
 ObjectState(observed=True, timestep=1, position=(1514.403955126936, -1215.4765045129302), heading=2.7782476807769734, velocity=(-12.307088723510377, 4.677199214156245)),
 ObjectState(observed=True, timestep=2, position=(1513.5738215903039, -1215.1680550506903), heading=2.7785522575709924, velocity=(-12.306393235459426, 4.692237664918388)),
 ObjectState(observed=True, timestep=3, position=(1512.6104514288609, -1214.8103705033695), heading=2.7790718841732693, velocity=(-12.307019122889981, 4.693907204978763)),
 ObjectState(observed=True, timestep=4, position=(1511.52504088425, -1214.4058783224682), heading=2.779450371792981, velocity=(-12.31029833732354, 4.674579002850383)),
 ObjectState(observed=True, timestep=5, position=(1510.3380908833, -1213.9617133792474), heading=2.77973212382364, velocity=(-12.311628460079117, 4

In [ ]:
import torch
from torch_geometric.nn import GATConv
from torch.nn import TransformerDecoder, TransformerDecoderLayer
from torch_geometric.data import DataLoader

class GNNEncoder(torch.nn.Module):
    def __init__(self, node_feature_dim, edge_feature_dim, hidden_dim):
        super().__init__()
        self.gnn1 = GATConv(node_feature_dim, hidden_dim)
        self.gnn2 = GATConv(hidden_dim, hidden_dim)

    def forward(self, x, edge_index):
        x = self.gnn1(x, edge_index).relu()
        x = self.gnn2(x, edge_index)
        return x

class TrajectoryDecoder(torch.nn.Module):
    def __init__(self, hidden_dim, num_tokens, num_layers=2):
        super().__init__()
        self.embedding = torch.nn.Embedding(num_tokens, hidden_dim)
        self.decoder_layer = TransformerDecoderLayer(d_model=hidden_dim, nhead=4)
        self.decoder = TransformerDecoder(self.decoder_layer, num_layers=num_layers)
        self.output_layer = torch.nn.Linear(hidden_dim, num_tokens)

    def forward(self, context, target_tokens):
        target_embeddings = self.embedding(target_tokens)
        decoded = self.decoder(target_embeddings, context)
        return self.output_layer(decoded)

class MotionModel(torch.nn.Module):
    def __init__(self, node_feature_dim, edge_feature_dim, hidden_dim, num_tokens):
        super().__init__()
        self.encoder = GNNEncoder(node_feature_dim, edge_feature_dim, hidden_dim)
        self.decoder = TrajectoryDecoder(hidden_dim, num_tokens)

    def forward(self, graph, target_tokens):
        context = self.encoder(graph.x, graph.edge_index)
        output = self.decoder(context, target_tokens)
        return output

# Example training loop
model = MotionModel(node_feature_dim=10, edge_feature_dim=5, hidden_dim=128, num_tokens=169)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for batch in DataLoader(train_dataset, batch_size=32, shuffle=True):
    optimizer.zero_grad()
    predictions = model(batch, batch.target_tokens)
    loss = torch.nn.CrossEntropyLoss()(predictions.view(-1, 169), batch.target_tokens.view(-1))
    loss.backward()
    optimizer.step()
